In [2]:
#https://population.un.org/wpp/Download/Standard/CSV/
#https://github.com/novelcovid/api

import pandas as pd
import json
import requests


### Fetching countries geo-data from corona.lmao.ninja's API

In [5]:
url = "https://corona.lmao.ninja/countries?sort=country"
req = requests.get(url)
j = json.loads(req.text)

# getting countries names for reference
countries = []
df = pd.DataFrame()
for country in j:
    countries.append(country)

df['country'] = pd.Series(countries)

# Parsing countries data do dict, then pandas dataframe
dic = []
i = 0
for country in countries:
    row = {'country': j[i]['country'], '_id': j[i]['countryInfo']['_id'], 'iso2': j[i]['countryInfo']['iso2'], 'iso3': j[i]['countryInfo']['iso3'],  
       'lat':j[i]['countryInfo']['lat'], 'long': j[i]['countryInfo']['long'], 'flag':j[i]['countryInfo']['flag']}
                 
    dic.append(row)
    i += 1 
df = pd.DataFrame.from_dict(dic)
df.tail()

,country,_id,iso2,iso3,lat,long,flag
205,Angola,24.0,AO,AGO,-12.5,18.5,https://raw.githubusercontent.com/NovelCOVID/A...
206,Andorra,20.0,AD,AND,42.5,1.6,https://raw.githubusercontent.com/NovelCOVID/A...
207,Algeria,12.0,DZ,DZA,28.0,3.0,https://raw.githubusercontent.com/NovelCOVID/A...
208,Albania,8.0,AL,ALB,41.0,20.0,https://raw.githubusercontent.com/NovelCOVID/A...
209,Afghanistan,4.0,AF,AFG,33.0,65.0,https://raw.githubusercontent.com/NovelCOVID/A...


### Fecthing countries data from UN's API
#### 2019 data

In [8]:
df_un= pd.read_csv('data/WPP2019_TotalPopulationBySex.csv')
df_un = df_un[df_un['Time']==2019]
df_un.tail()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
277314,900,World,2,Medium,2019,2019.5,3889034.611,3824433.594,7713468.205,59.291
278198,1801,World Bank Regional Groups (developing only),2,Medium,2019,2019.5,3261387.881,3191129.174,6452517.055,70.258
278349,887,Yemen,2,Medium,2019,2019.5,14692.284,14469.638,29161.922,55.234
279233,894,Zambia,2,Medium,2019,2019.5,8843.214,9017.820,17861.034,24.026
280117,716,Zimbabwe,2,Medium,2019,2019.5,6983.353,7662.120,14645.473,37.858


In [111]:
df_un[df_un['Location']=='Brazil']

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
30795,76,Brazil,2,Medium,2019,2019.5,103733.16,107316.359,211049.519,25.251


#### Merging both dataframes and saving to csv

In [9]:
df_countries = pd.merge(df, df_un, how='inner', on=None, left_on='country', 
                 right_on='Location', left_index=False, right_index=False, sort=False)   

df_countries.drop(['_id', 'Location', 'MidPeriod'], axis=1, inplace=True)
df_countries.tail()

,country,iso2,iso3,lat,long,flag,LocID,VarID,Variant,Time,PopMale,PopFemale,PopTotal,PopDensity
176,Angola,AO,AGO,-12.5,18.5,https://raw.githubusercontent.com/NovelCOVID/A...,24,2,Medium,2019,15744.779,16080.520,31825.299,25.528
177,Andorra,AD,AND,42.5,1.6,https://raw.githubusercontent.com/NovelCOVID/A...,20,2,Medium,2019,NaN,NaN,77.146,164.140
178,Algeria,DZ,DZA,28.0,3.0,https://raw.githubusercontent.com/NovelCOVID/A...,12,2,Medium,2019,21749.666,21303.388,43053.054,18.076
179,Albania,AL,ALB,41.0,20.0,https://raw.githubusercontent.com/NovelCOVID/A...,8,2,Medium,2019,1466.785,1414.128,2880.913,105.143
180,Afghanistan,AF,AFG,33.0,65.0,https://raw.githubusercontent.com/NovelCOVID/A...,4,2,Medium,2019,19529.727,18512.030,38041.757,58.269


In [12]:
df_countries.to_csv('data/world_countries_2019.csv',  index = False)